In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 20160510: both versions of BkAvgAst created (nohist & ultcert)
###           used Tickers from earlier version, then supplemented with missing Tickers if aeTotAst>800bn
### 20170407: updated for 2016-Dec updated FailedBank list and 2017-Mar institutions2.csv file

import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201712.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2017-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2017-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2017-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2017-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.440068e+09  1.414389e+09  5.003771e+08  6.485640e+08   
1 2001-03-31  1.240240e+09  1.244810e+09  6.494579e+08  7.795060e+08   
2 2001-03-31  1.043060e+09  1.040029e+09  5.003295e+08  2.733820e+08   
3 2001-03-31  8.459902e+08  8.336269e+08  5.148696e+08  5.818909e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.533041e+08  9.018681e+08        NaT  9.183039   9.183039  5.605613   
1  1.066504e+08  8.861565e+08 2009-01-16  7.908771  17.091809  7.275732   
2  4.369433e+08  7.103253e+08 2008-11-23  6.651396  23.743205  5.605080   
3  6.797765e+07  6.498685e+08        NaT  5.394718  29.137923  5.767969   
4  2.469325e+07  2.951088e+08        NaT  2.816295  31.954217  2.852641   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.605613  7.265718   7.265718  
1  12.881345  8.732632  15.998350  
2  18.486425  3.062638  19.060987  
3  24.254394  6.518793  25.579781  
4  27.107035  3.029405  28.609186

In [3]:
len(rk) # 10767

10767

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_201802.csv", encoding='latin-1')
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street     48,570      NM   
1  Connecticut     6       0       0  81 West Main Street    624,655      NM   
2        Maine     8       0       0      One City Center  1,699,404      SM   
3        Maine     9    9781       0       66 Main Street    539,169      SM   
4        Maine    10       0       0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   
2       223         0         0 ...               Portland-South Portland, ME   
3       223         0         0 ...                                       NaN   
4       223         0         0 ...               Portland-South Portland, ME   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   
2    38860               Portland-South Portland, ME      38860   
3        0                                       NaN          0   
4    38860               Portland-South Portland, ME      38860   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  
2              1              0       NaN            0            0 NaN  
3              0              0       NaN            0            0 NaN  
4              1              0       NaN            0            0 NaN  

[5 rows x 133 columns]

In [5]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [6]:
len(bhc) # 27602

27602

In [7]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street   48,570      NM   
1  Connecticut     6       0       0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  

[2 rows x 133 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
len(bhc2) # 27602

27602

In [9]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [10]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635                           AuburnBank  AL      SM      8.0   
19    39  430036            Robertson Banking Company  AL      NM      6.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      3.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA       14  
15        Auburn National Bancorp Inc  1129533.0    AL       35  
19                            Rbc Inc  2526755.0    AL       39  
21       Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Fin Services Inc  1082955.0    AL       46  
26   Traders & Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares IncESOPlan  3587294.0    AL       50  
30        First Vernon Bancshares Inc  1139998.0    AL       51  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL       52  
33                       Gadsden Corp  1079067.0    AL       54

In [11]:
bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

Empty DataFrame
Columns: [Cert, IDRSSD, Name, ST, BkClass, Offices, Parent, ParRSSD, ParST, UltCert]
Index: []

In [12]:
bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [13]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612.csv", index_col=0)
#rkhn = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612n.csv", index_col=0)

#len(rkh), len(rkhn) # 10762
len(rkh) # 10762

10762

In [14]:
rkh.columns

Index([u'Rank', u'IDRSSD', u'xAsOf', u'xName', u'xState', u'xCert', u'nAsOf',
       u'FailDate', u'BkClass', u'Offices', u'Parent', u'ParRSSD', u'ParST',
       u'UltCert', u'Ticker', u'aeTotAst', u'aaTotAst', u'aeTotLn', u'aeDDep',
       u'aeFDep', u'aeTotDep', u'pr_eTA', u'pr_eTAcs', u'pr_eTL', u'pr_eTLcs',
       u'pr_eDD', u'pr_eDDcs'],
      dtype='object')

In [15]:
len(rkh[rkh['Ticker']>'']) # count before new BHCs added: 2932

2932

In [16]:
#len(rkhn[rkhn['Ticker']>'']) # count after new BHCs added

In [17]:
#rkhn[rkhn['Ticker']=='BFIN'] # this will be added by missing ticker file (later)

In [18]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 8672

8672

In [19]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10758  10759  1094314.0    MO  CBCYB
10761  10762  3447567.0    TX    NaN

In [20]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 416

416

In [21]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [22]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [23]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACNB          1
ADKT          1

In [24]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [25]:
bhc2.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [26]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [27]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4595.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [28]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4595.0  Bank Of America Corp  1073757.0    NC     3510

In [29]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2017-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2017-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eTLcs    pr_eDD  \
0  1.440068e+09  1.414389e+09  5.003771e+08   ...      5.605613  7.265718   
1  1.240240e+09  1.244810e+09  6.494579e+08   ...     12.881345  8.732632   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.265718   628.0        N   5287.0  Jpmorgan Chase & Co.  1039502.0   
1  15.998350  3510.0        N   4595.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [30]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [31]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2017-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2017-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eDD   pr_eDDcs  \
0  1.440068e+09  1.414389e+09  5.003771e+08   ...    7.265718   7.265718   
1  1.240240e+09  1.244810e+09  6.494579e+08   ...    8.732632  15.998350   

     Cert BkClass  Offices                Parent    ParRSSD  ParST  UltCert  \
0   628.0       N   5287.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0   
1  3510.0       N   4595.0  Bank Of America Corp  1073757.0     NC   3510.0   

   Ticker  
0     JPM  
1     BAC  

[2 rows x 28 columns]

In [32]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [33]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2017-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2017-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices    ...           aeTotLn        aeDDep  \
0        NaT       N   5287.0    ...      5.003771e+08  6.485640e+08   
1 2009-01-16       N   4595.0    ...      6.494579e+08  7.795060e+08   

         aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
0  2.533041e+08  9.018681e+08  9.183039   9.183039  5.605613   5.605613   
1  1.066504e+08  8.861565e+08  7.908771  17.091809  7.275732  12.881345   

     pr_eDD   pr_eDDcs  
0  7.265718   7.265718  
1  8.732632  15.998350  

[2 rows x 27 columns]

In [34]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201712.csv")

In [35]:
len(rk4) # 10767

10767

In [36]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 213

213

In [37]:
rkm.head()

Rank   IDRSSD      xAsOf                         xName xState    xCert  \
143   144   527954 2017-12-31  FIRST NATIONAL BANK OF OMAHA     NE   5452.0   
149   150   311845 2017-12-31                   ARVEST BANK     AR   8728.0   
165   166  1842065 2017-12-31                 CIBC BANK USA     IL  33306.0   
169   170   137915 2017-12-31                 EMIGRANT BANK     NY  12054.0   
182   183   877369 2017-12-31                  RABOBANK, NA     CA  23364.0   

         nAsOf FailDate BkClass  Offices    ...           aeTotLn  \
143 2001-03-31      NaT       N    127.0    ...      7.491336e+06   
149 2001-03-31      NaT      SM    268.0    ...      6.673750e+06   
165 2001-03-31      NaT      NM     27.0    ...      6.923300e+06   
169 2001-03-31      NaT      SB      5.0    ...      5.161373e+06   
182 2001-03-31      NaT       N    107.0    ...      5.946541e+06   

           aeDDep       aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
143  8.817051e+06     0.000000  8.817051e+06  0.070830  73.865823  0.083924   
149  8.878200e+06  3384.470588  8.881584e+06  0.067222  74.277497  0.074765   
165  7.674912e+06     0.000000  7.674912e+06  0.059063  75.271141  0.077560   
169  6.299948e+06     0.000000  6.299948e+06  0.057890  75.503457  0.057822   
182  5.952796e+06     0.000000  5.952796e+06  0.054542  76.225888  0.066618   

      pr_eTLcs    pr_eDD   pr_eDDcs  
143  69.988271  0.098775  73.033152  
149  70.442424  0.099460  73.575238  
165  71.445399  0.085980  74.820627  
169  71.716053  0.070577  75.117033  
182  72.569761  0.066688  75.925200  

[5 rows x 27 columns]

In [38]:
rkm.to_csv('rk_missingtickers201712.csv')

In [40]:
## start up here after filling in missing tickers

In [41]:
## filled in many missing tickers

In [42]:
rkm2 = pd.read_csv('rk_missingtickers201712a.csv',index_col=0)
rkm2.head()

Rank   IDRSSD       xAsOf                         xName xState  xCert  \
143   144   527954  12/31/2017  FIRST NATIONAL BANK OF OMAHA     NE   5452   
149   150   311845  12/31/2017                   ARVEST BANK     AR   8728   
165   166  1842065  12/31/2017                 CIBC BANK USA     IL  33306   
169   170   137915  12/31/2017                 EMIGRANT BANK     NY  12054   
182   183   877369  12/31/2017                  RABOBANK, NA     CA  23364   

         nAsOf FailDate BkClass  Offices    ...          aeTotLn       aeDDep  \
143  3/31/2001      NaN       N      127    ...      7491336.059  8817051.088   
149  3/31/2001      NaN      SM      268    ...      6673749.882  8878199.721   
165  3/31/2001      NaN      NM       27    ...      6923299.853  7674911.838   
169  3/31/2001      NaN      SB        5    ...      5161372.985  6299948.294   
182  3/31/2001      NaN       N      107    ...      5946540.794  5952795.735   

          aeFDep     aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
143     0.000000  8817051.088  0.070830  73.865823  0.083924  69.988271   
149  3384.470588  8881584.191  0.067222  74.277497  0.074765  70.442424   
165     0.000000  7674911.838  0.059063  75.271141  0.077560  71.445399   
169     0.000000  6299948.294  0.057890  75.503457  0.057822  71.716053   
182     0.000000  5952795.735  0.054542  76.225888  0.066618  72.569761   

       pr_eDD   pr_eDDcs  
143  0.098775  73.033152  
149  0.099460  73.575238  
165  0.085980  74.820627  
169  0.070577  75.117033  
182  0.066688  75.925200  

[5 rows x 27 columns]

In [43]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
165  1231342     CM
207  4037349   CADE
450  1085013   SBCF
472  3831250   UCFC
661  4037349   CADE

In [44]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 19, 18

(19, 18)

In [45]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName  \
165                              CIBC BANK USA   
207                         CADENCE BANK, N.A.   
450                     SEACOAST NATIONAL BANK   
472                          HOME SAVINGS BANK   
661                         CADENCE BANK, N.A.   
812                      FRANKLIN SYNERGY BANK   
895   SAVINGS INSTITUTE BANK AND TRUST COMPANY   
935                                  PCSB BANK   
1036                              CAPSTAR BANK   
1053                    BLACKHAWK BANK & TRUST   
1066                      BELMONT SAVINGS BANK   
1100                       ATLANTIC COAST BANK   
1106   PEOPLES SECURITY BANK AND TRUST COMPANY   
1111                     NICOLET NATIONAL BANK   
1196                       ROYAL BUSINESS BANK   
1234                                  SONABANK   
1235                        FIRST SAVINGS BANK   
1257                 NATIONAL BANK OF COMMERCE   
1274                              CRESCOM BANK   
4148                      PREMIER BANK & TRUST   
4780                ATLANTIC TRUST COMPANY, NA   

                                         Parent Ticker  
165          Canadian Imperial Bank Of Commerce    NaN  
207                         Cadence Bancorp Llc    NaN  
450            Seacoast Banking Corp Of Florida    NaN  
472                   United Community Fin Corp    NaN  
661                         Cadence Bancorp Llc    NaN  
812                    Franklin Fin Network Inc    NaN  
895                            Si Fin Group Inc    NaN  
935                               Pcsb Fin Corp    NaN  
1036                   Capstar Fin Holdings Inc    NaN  
1053                      Blackhawk Bancorp Inc    NaN  
1066                            Bsb Bancorp Inc    NaN  
1100                    Atlantic Coast Fin Corp    NaN  
1106                  Peoples Fin Services Corp    NaN  
1111                     Nicolet Bankshares Inc    NaN  
1196                                Rbb Bancorp    NaN  
1234  Southern National Bancorp Of Virginia Inc    NaN  
1235                First Savings Fin Group Inc    NaN  
1257                     National Commerce Corp    NaN  
1274                          Carolina Fin Corp    NaN  
4148                  United Community Fin Corp    NaN  
4780         Canadian Imperial Bank Of Commerce    NaN

In [46]:
rk4c = rk4.copy()

In [47]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1085013: 'SBCF',
 1139541: 'PFIS',
 1231342: 'CM',
 1823345: 'BHWB',
 2507790: 'CARO',
 3103603: 'NCBS',
 3306815: 'SIFI',
 3325740: 'SONA',
 3551389: 'NCOM',
 3637582: 'FSB',
 3831250: 'UCFC',
 3840029: 'ACFC',
 3908929: 'FSFG',
 3929791: 'BLMT',
 4037349: 'CADE',
 4176855: 'RBB',
 4933272: 'CSTR',
 5079476: 'PCSB'}

In [48]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [49]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName  \
165                              CIBC BANK USA   
207                         CADENCE BANK, N.A.   
450                     SEACOAST NATIONAL BANK   
472                          HOME SAVINGS BANK   
661                         CADENCE BANK, N.A.   
812                      FRANKLIN SYNERGY BANK   
895   SAVINGS INSTITUTE BANK AND TRUST COMPANY   
935                                  PCSB BANK   
1036                              CAPSTAR BANK   
1053                    BLACKHAWK BANK & TRUST   
1066                      BELMONT SAVINGS BANK   
1100                       ATLANTIC COAST BANK   
1106   PEOPLES SECURITY BANK AND TRUST COMPANY   
1111                     NICOLET NATIONAL BANK   
1196                       ROYAL BUSINESS BANK   
1234                                  SONABANK   
1235                        FIRST SAVINGS BANK   
1257                 NATIONAL BANK OF COMMERCE   
1274                              CRESCOM BANK   
4148                      PREMIER BANK & TRUST   
4780                ATLANTIC TRUST COMPANY, NA   

                                         Parent Ticker  
165          Canadian Imperial Bank Of Commerce     CM  
207                         Cadence Bancorp Llc   CADE  
450            Seacoast Banking Corp Of Florida   SBCF  
472                   United Community Fin Corp   UCFC  
661                         Cadence Bancorp Llc   CADE  
812                    Franklin Fin Network Inc    FSB  
895                            Si Fin Group Inc   SIFI  
935                               Pcsb Fin Corp   PCSB  
1036                   Capstar Fin Holdings Inc   CSTR  
1053                      Blackhawk Bancorp Inc   BHWB  
1066                            Bsb Bancorp Inc   BLMT  
1100                    Atlantic Coast Fin Corp   ACFC  
1106                  Peoples Fin Services Corp   PFIS  
1111                     Nicolet Bankshares Inc   NCBS  
1196                                Rbb Bancorp    RBB  
1234  Southern National Bancorp Of Virginia Inc   SONA  
1235                First Savings Fin Group Inc   FSFG  
1257                     National Commerce Corp   NCOM  
1274                          Carolina Fin Corp   CARO  
4148                  United Community Fin Corp   UCFC  
4780         Canadian Imperial Bank Of Commerce     CM

In [50]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 525, 504

(525, 504)

In [51]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201712.csv")

In [52]:
## process ultcert version:

In [53]:
rk5 = rk4.copy()

In [54]:
ult = set(rk5['UltCert'])
len(ult) # 5985

5985

In [55]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [56]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4379

4379

In [57]:
len(bhcc[bhcc['Ticker']>'']) # 511

511

In [58]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [59]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
39     16571.0                 Bank Of Montreal    BMO  1231333.0     0
207     4999.0              Cadence Bancorp Llc   CADE  4037349.0    TX
254    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
472    35385.0        United Community Fin Corp   UCFC  3831250.0    OH
504    26342.0             Royal Bank Of Canada     RY  1232497.0     0
661     4999.0              Cadence Bancorp Llc   CADE  4037349.0    TX
747    35525.0    Bankcap Partners Fund I, L.P.    NaN  3559657.0    TX
969    35525.0    Bankcap Partners Fund I, L.P.    NaN  3559657.0    TX
1090    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1183   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1232    3857.0     Bear State Fin Holdings, Llc    NaN  4297853.0    AR
1660    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
1666   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1729   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1771      89.0   White River Bancshares Company    NaN  3306589.0    AR
2071   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
2140   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2418   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
2600   26342.0             Royal Bank Of Canada     RY  1232497.0     0
3250   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4043   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4148   35385.0        United Community Fin Corp   UCFC  3831250.0    OH
4275   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4414   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
4444    3857.0     Bear State Fin Holdings, Llc    NaN  4297853.0    AR
4765   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
4870   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
5297    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
5902    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6821   17399.0                   First Fin Corp    NaN  1067514.0    KS
7032      89.0   White River Bancshares Company    NaN  3306589.0    AR
7040   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
7420   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7440   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7566   15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
7706    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
7892    6080.0    John T. Vucurevich Foundation    NaN  3613504.0    SD
8728   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
9082   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9367    6080.0    John T. Vucurevich Foundation    NaN  3613504.0    SD
9639    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9764   17399.0                   First Fin Corp    NaN  1067514.0    KS
10143  15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
10182  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10219  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10504  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [60]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6821  17399.0  First Fin Corp    NaN  1067514.0    KS
9764  17399.0  First Fin Corp    NaN  1067514.0    KS

In [61]:
len(bhcc) # 4379

4379

In [62]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4333

4333

In [63]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [64]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [65]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [66]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

u'Jpmorgan Chase & Co.'

In [67]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [68]:
len(rk5c[rk5c['Ticker']>'']) # 3049

3049

In [69]:
j # 4266

4266

In [70]:
len(rk5c) # 10767

10767

In [71]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult201712.csv")

In [72]:
rk4[rk4['Ticker']=='ZION']  ## current active affiliate (1 bank list)

Rank  IDRSSD      xAsOf   xName xState   xCert      nAsOf FailDate  \
85    86  276579 2017-12-31  ZB, NA     UT  2270.0 2001-03-31      NaT   

   BkClass  Offices    ...           aeTotLn        aeDDep         aeFDep  \
85       N    441.0    ...      1.500349e+07  1.610787e+07  901032.044118   

        aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
85  1.700890e+07  0.136881  68.121028  0.168081  63.555369  0.180453   

     pr_eDDcs  
85  66.421935  

[1 rows x 27 columns]

In [73]:
rk5c[rk5c['Ticker']=='ZION']  ## all acquired & consolidated banks (23 banks)

Rank   IDRSSD      xAsOf                                 xName xState  \
85      86   276579 2017-12-31                                ZB, NA     UT   
150    151   837260 2015-09-30               CALIFORNIA BANK & TRUST     CA   
155    156   676656 2015-09-30                        AMEGY BANK, NA     TX   
296    297  1004368 2015-09-30              NATIONAL BANK OF ARIZONA     AZ   
325    326   456960 2015-09-30                     NEVADA STATE BANK     NV   
424    425   933957 2015-09-30              VECTRA BANK COLORADO, NA     NM   
696    697   807665 2009-06-30                     VINEYARD BANK, NA     CA   
1071  1072   458263 2006-12-31                  STOCKMEN'S BANK, THE     AZ   
1118  1119  1198641 2015-09-30  COMMERCE BANK OF WASHINGTON, NA, THE     WA   
1180  1181  2479310 2008-06-30                     SILVER STATE BANK     NV   
1593  1594   720465 2008-12-31                         ALLIANCE BANK     CA   
1657  1658   618656 2004-09-30                            KLEIN BANK     TX   
2686  2687   105950 2001-09-30          MINNEQUA BANK OF PUEBLO, THE     CO   
3148  3149   576158 2003-06-30                             MAXIMBANK     TX   
4059  4060   309664 2006-06-30                           RANCHO BANK     CA   
4140  4141   521550 2003-12-31                        LONE STAR BANK     TX   
4619  4620  2068134 2009-03-31            GREAT BASIN BANK OF NEVADA     NV   
6162  6163   626754 2001-03-31              LEAGUE CITY BANK & TRUST     TX   
6340  6341  3161993 2008-03-31                           CHOICE BANK     AZ   
6639  6640   687269 2007-06-30        INTERCONTINENTAL NATIONAL BANK     TX   
6671  6672   456670 2002-09-30                   FRONTIER STATE BANK     AZ   
8388  8389  3374186 2015-03-31          COMMERCE BANK OF OREGON, THE     OR   
9883  9884  2907448 2003-06-30               SOUTHLAND BUSINESS BANK     CA   

        xCert      nAsOf   FailDate BkClass  Offices     ...      \
85     2270.0 2001-03-31        NaT       N    441.0     ...       
150   20852.0 2001-03-31        NaT      NM     99.0     ...       
155   24107.0 2001-03-31        NaT       N     81.0     ...       
296   20626.0 2001-03-31        NaT       N     66.0     ...       
325   18113.0 2001-03-31        NaT      NM     50.0     ...       
424    2993.0 2001-03-31        NaT       N     37.0     ...       
696   23556.0 2001-03-31 2009-07-17       N     17.0     ...       
1071  23182.0 2001-03-31        NaT      NM     43.0     ...       
1118  27298.0 2001-03-31        NaT       N      2.0     ...       
1180  34194.0 2001-03-31 2008-09-05      NM     18.0     ...       
1593  23124.0 2001-03-31 2009-02-06      NM      5.0     ...       
1657  22866.0 2001-03-31        NaT      NM     27.0     ...       
2686   1773.0 2001-03-31        NaT      NM      5.0     ...       
3148   9017.0 2001-03-31        NaT      NM      9.0     ...       
4059  22934.0 2001-03-31        NaT      NM      4.0     ...       
4140  22304.0 2001-03-31        NaT      NM      6.0     ...       
4619  33824.0 2001-03-31 2009-04-17      NM      5.0     ...       
6162  17577.0 2001-03-31        NaT      NM      4.0     ...       
6340  57469.0 2003-03-31        NaT      NM      3.0     ...       
6639  19112.0 2001-03-31        NaT       N      3.0     ...       
6671  24227.0 2001-03-31        NaT      NM      7.0     ...       
8388  58223.0 2005-12-31        NaT      NM      1.0     ...       
9883  35287.0 2001-03-31        NaT      NM      1.0     ...       

           aeTotLn        aeDDep         aeFDep      aeTotDep    pr_eTA  \
85    1.500349e+07  1.610787e+07  901032.044118  1.700890e+07  0.136881   
150   7.675023e+06  8.519186e+06   32194.661017  8.551380e+06  0.065761   
155   6.722988e+06  7.001789e+06  646260.186441  7.648049e+06  0.062903   
296   3.325315e+06  3.496657e+06    2862.949153  3.499520e+06  0.026797   
325   2.461972e+06  3.219857e+06       0.000000  3.219857e+06  0.023575   
424   1.905816e+06  1.9167